In [ ]:
from skimage.segmentation import watershed
import numpy as np
import cv2
from copy import deepcopy
from scipy import ndimage as ndi
import matplotlib.pyplot as plt


def count_big_bubbles(img1, min_size = 150):
  min_area = min_size
  gray_im = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(gray_im,180,255,0)  
  #plt.imshow(thresh)
  markers = ndi.label(thresh)[0]
  d = []
  ids, areas = np.unique(markers, return_counts=True)
  for id, area in zip(ids, areas):
    if area < min_area:
      d.append(id)
  #print(d)
  big_markers = deepcopy(markers)
  for id in d:
    big_markers[markers == id] = 0
  return np.unique(big_markers).shape[0]

In [ ]:
import cv2
import numpy as np
import os
import glob
from wshed import watershed_segmentation
from collections import deque
#from funcutils import *
import json
import matplotlib.pyplot as plt


feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.5,
                       minDistance = 30,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (30,30),
                  maxLevel = 4,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))



dir_path = "./data/task1-data-part2"
# filenames = glob.glob(os.path.join(dir_path + "2", "*.ts"))  # temporary path
filenames = ['/content/F1_2_4_1.ts']
counter = 1
for filename in filenames:
    #filename = os.path.join(dir_path, filename)
    #print(filename[:-3])
    cap = cv2.VideoCapture(filename)
    fourcc = cv2.VideoWriter_fourcc((*'XVID'))
    out = cv2.VideoWriter(filename[:-3] + '.mp4', fourcc, 30.0, (800, 600))

    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
    frame_n = 0
    kernel_open = np.ones((7,7),np.uint8)  # needs to be tuned
    kernel_close = np.ones((33, 33), np.uint8)

    queue = deque(maxlen=30)

    i = 0  # dummy counter
    row = []
    f = 0

    #tracking 
    pTime = 0
    pts = deque(maxlen=10)
    ret, old_frame = cap.read()
    old_frame = deepcopy(old_frame[:300, :, :])
    old_gray = cv2.cvtColor(old_frame[:300,:, :], cv2.COLOR_BGR2GRAY)
    #print(old_gray.shape)
    #print(old_frame.shape)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    #print(frame)
    old_bn = count_big_bubbles(old_frame, min_size = 250)
    mask_tracking = np.zeros_like(old_frame)
    max_difference = 0
    average_speed = 0
    frame_p = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-40
    for qqq in range(frame_p-40):
      if qqq < 99999999:
          ret, frame = cap.read()
          final_frame = deepcopy(frame)
          if not ret:
              print('No ret detected!')
              break
          new_bn = count_big_bubbles(frame)
          max_difference = max([abs(new_bn - old_bn), max_difference])
          #tracking
          frame_tracking = deepcopy(frame)
          frame_tracking = frame_tracking[:300, :, :]
          frame_gray = cv2.cvtColor(frame_tracking, cv2.COLOR_BGR2GRAY)
          counter += 1
          if counter % 10 == 0:
              mask_tracking = np.zeros_like(old_frame)
              p0_update = cv2.goodFeaturesToTrack(frame_gray, mask = None, **feature_params)
              #print(p0)
              #print(p0_update)
              #print(p0.shape)
              #print(p0_update.shape)
              if not p0_update is None:
                p0 = np.concatenate((p0, p0_update), axis=0)
          #print(old_gray.shape)
          #print(frame_gray.shape)
          p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
          good_new = p1[st==1]
          good_old = p0[st==1]
          c_xy = 0
          dX = []
          dY = []
          average_speed = []
          for i,(new,old) in enumerate(zip(good_new,good_old)):
              a,b = new.ravel()
              c,d = old.ravel()
              dX.append(a - c)
              dY.append(b - d)
              new_mask_tracking = cv2.line(mask_tracking, (a,b),(c,d), (255,0,255), 2)
              frame_tracking = cv2.circle(frame_tracking,(a,b),5,(255,0,255),-1)
          cur_speed = np.sqrt(np.mean(dX) ** 2 + np.mean(dY) ** 2) * 30
          average_speed.append(cur_speed)
          #img = mask
          #img = cv2.add(frame,new_mask)
          mask_tracking = new_mask_tracking
          #plt.imshow(mask)
          #cv2.imshow('frame',img)
          #img = cv2.resize(img, (800, 600), cv2.INTER_LANCZOS4)
          #out.write(img)
          old_gray = frame_gray.copy()
          p0 = good_new.reshape(-1,1,2)
          counter+=1

          #end tracking
          frame = cv2.resize(frame, (0,0), fx=1, fy=1)
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          gray = cv2.equalizeHist(gray)



          if frame_n < 1:  # create for the first frame only
              canvas = np.zeros(gray.shape).astype(np.uint8)
          elif frame_n < 50:#600:  # collect mask for the first 1000 frames
              fgmask = fgbg.apply(gray)  # get only moving pixels mask
              canvas = (canvas + fgmask).astype(np.uint8)
              f = 0
              #plt.imshow(fgmask)
          if f == 0:
            canvas[canvas > 0] = 255
            opening = cv2.morphologyEx(255 - canvas, cv2.MORPH_OPEN, kernel_open, iterations=2)  # opening
            closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_close, iterations=2)  # closing
            wshed = watershed_segmentation(frame)
            final_frame[wshed == 0] = (255, 0, 0)
            #print(new_mask_tracking.shape)
            final_frame = cv2.add(final_frame, np.concatenate((new_mask_tracking, np.zeros(shape = (300, 800, 3), dtype = np.uint8)), axis = 0))
            res = cv2.bitwise_and(final_frame, final_frame, mask=(255 - closing))
            gray_res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)  # without watershed algo
            wshed = cv2.bitwise_and(wshed.astype(np.uint8), wshed.astype(np.uint8), mask=(255-closing))
            values, areas = np.unique(wshed, return_counts=True)
            queue.appendleft(np.mean(areas))

            if frame_n % 30 == 0:
                #print(np.mean(queue))
                for_json = {
                    'area': np.mean(queue),
                    'stability': 10/(0.35*max_difference+1),
                    'speed': sum(average_speed)/(len(average_speed)+1)

                }
                average_speed = 0
                max_difference = 0
                row.append(for_json)
                with open(f"{filename[:-3]}" + ".json", 'w') as outfile:
                    json.dump(row, outfile)
                print(frame_n)

            #cv2.imshow('frame', res)

            out.write(res)
            #if frame_n > 100:
              #break
            frame_n += 1

            i += 1
    out.release()

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
No ret detected!


In [ ]:
import shutil
shutil.move('/content/F1_2_4_1.mp4','/content/drive/MyDrive')

'/content/drive/MyDrive/F1_2_4_1.mp4'